# Tagging lab
## Viterbi
$k = 2$
The Markov assumption:
$$
\mathbb{P}(w_1, w_2, \ldots w_n \ | \ l_1, l_2 \ldots l_n) =
    \prod_{i=1}^n\mathbb{P}(l_i \ | l_{i-1})\cdot\mathbb{P}(w_i \ | \ l_i)
$$
The Viterbi function gives the optimal value:
$$
V(w_1, w_2, \ldots w_n) =
{\arg\max}_{l_i\in L}
    \prod_{i=1}^n\mathbb{P}(l_i \ | l_{i-1})\cdot\mathbb{P}(w_i \ | \ l_i)
$$
For small $n$-s:
$$
V(w_1) = {\arg\max}_{l_1\in L} \mathbb{P}(w_1 \ | \ l_1)
$$

$$
V(w_1, w_2) = {\arg\max}_{l_1, l_2\in L} \mathbb{P}(w_1 \ | \ l_1) \cdot \mathbb{P}(l_2 \ | \ l_1) \cdot \mathbb{P}(w_2 \ | \ l_2)
$$

$$
V(w_1, w_2, w_3) = {\arg\max}_{l_1, l_2, l_3\in L} V(w_1, w_2) \cdot \mathbb{P}(l_3 \ | \ l_2) \cdot \mathbb{P}(w_3 \ | \ l_3)
$$

$$
V(w_1, w_2, w_3, w_4) = {\arg\max}_{l_1, l_2, l_3, l_4\in L} V(w_1, w_2, w_3) \cdot \mathbb{P}(l_4 \ | \ l_3) \cdot \mathbb{P}(w_4 \ | \ l_4)
$$

## Implementing Viterbi
For $k=2$
### Task 1
Read the file [`umbc.casesensitive.word_pos.1M.txt`](http://sandbox.hlt.bme.hu/~gaebor/ea_anyag/python_nlp) line-by-line and make a vocabulary of words and labels. The file is in a format:

`"word\tpos\n"`
You have to have:
* a dict of words to indices
* a dict of labels to indices
* reverse dict, indices to words

Note that there can be some errors in the file, character encoding and delimiter!

In [1]:
from collections import defaultdict

emissions = defaultdict(lambda: defaultdict(int))
transitions = defaultdict(lambda: defaultdict(int))

word_idx = defaultdict(lambda: len(word_idx))
pos_idx = defaultdict(lambda: len(pos_idx))

with open("umbc.casesensitive.word_pos.1M.txt", encoding="utf8") as f:
    first = next(f)
    word, pos = first.rstrip('\t').split('\t')
    word_idx[word]
    pos_idx[pos]
    emissions[pos][word] += 1
    for line in f:
        fd = line.rstrip('\n').split('\t')
        if len(fd) != 2:
            continue
        word = fd[0]
        new_pos = fd[1]
        emissions[new_pos][word] += 1
        transitions[pos][new_pos] += 1
        pos = new_pos
        word_idx[word]
        pos_idx[pos]

### Task 2
Obtain the following matrices:
* counts of words with pos tags
  * a $|V|\times |L|$ matrix of integers
  $$M(i,j) = \# \ i^\text{th} \text{ word with pos tag } j$$
  * an $|L|\times |L|$ matrix of integers
  $$N(i,j) = \# \ i^\text{th} \text{ pos after } j^\text{th} \text { pos}$$

After that
* empirical probabilities
  * a $|V|\times |L|$ matrix of floats
  $$P_1(i,j) = \frac{\# \ i^\text{th} \text{ word with pos tag } j}{\# \ \text{pos tag } j}$$
  * an $|L|\times |L|$ matrix of floats
  $$P_2(i,j) = \frac{\# \ i^\text{th} \text{ pos after } j^\text{th} \text { pos}}{\# \ \text{pos tag } j}$$


In [2]:
import numpy as np

em_probs = np.zeros((len(pos_idx), len(word_idx)), dtype="float32")

for pos, em_freqs in emissions.items():
    for word, cnt in em_freqs.items():
        em_probs[pos_idx[pos], word_idx[word]] = cnt
em_probs = em_probs / em_probs.sum(axis=1).reshape(em_probs.shape[0], 1)

In [3]:
tr_probs = np.zeros((len(pos_idx), len(pos_idx)), dtype="float32")

for pos1, tr in transitions.items():
    for pos2, cnt in tr.items():
        tr_probs[pos_idx[pos1], pos_idx[pos2]] = cnt
tr_probs = tr_probs / tr_probs.sum(axis=1).reshape(tr_probs.shape[0], 1)

### Task 3
Implement the pseudocode in the tagging lecture ($k=2$).
Use the global variables `P1` and `P2` (and the dictionaries).

It is useful to have a one-step-viterbi function.
$$
\mathrm{step}(\pi_{k-1}, v, word) = \max_{w\in L} \left(\pi(k−1,w)\cdot \mathbb{P}(v \ | \ w)\cdot \mathbb{P}(word \ | \ v)\right) 
$$
$\pi_{k-1}$ is a vector, the previous column of the dynamic table. 

In [4]:
P1 = em_probs
P2 = tr_probs

idx_word = {v: k for k, v in word_idx.items()}
idx_pos = {v: k for k, v in pos_idx.items()}

def viterbi_step(previous, v, w, word):
    return previous[pos_idx[w]] * P2[pos_idx[w], pos_idx[v]] * P1[pos_idx[v], word_idx[word]]

In [5]:
def viterbi(words):
    """from a list of strings returns the optimal probability"""
    V = np.zeros((len(pos_idx), len(words)), dtype="float64")
    backptr = np.zeros((len(pos_idx), len(words)), dtype="int8")
    w0 = word_idx[words[0]]
    V[:, 0] = P1[:, w0]
    backptr[:, 0] = 0
    
    for i in range(1, len(words)):
        word = words[i]
        for pos in pos_idx.keys():
            m = max((viterbi_step(V[:, i-1], pos, prev_pos, words[i]), prev_pos) for prev_pos in pos_idx.keys())
            V[pos_idx[pos], i] = m[0]
            backptr[pos_idx[pos], i] = pos_idx[m[1]]
    decoded = [idx_pos[np.argmax(V[:, -1])]]
    for i in range(len(words)-1, -1, -1):
        bp = backptr[pos_idx[decoded[-1]], i]
        decoded.append(idx_pos[bp])
    return decoded[::-1][1:]

s = "the dog runs ."
words = s.split() * 5
tags = viterbi(words)
print(len(tags))
tags[:10], tags[-10:]

20


(['DT', 'NN', 'NNS', '.', 'DT', 'NN', 'NNS', '.', 'DT', 'NN'],
 ['NNS', '.', 'DT', 'NN', 'NNS', '.', 'DT', 'NN', 'NNS', '.'])

### Task 4
Add the backtracking with an extra table, which stores the argmax, not the max value.

In [6]:
import numpy, scipy
from collections import defaultdict

In [7]:
with open("umbc.casesensitive.word_pos.1M.txt", "rb") as f:
    for line in f:
        parts = line.strip().split(b'\t')
        if len(parts) == 2:
            word = parts[0]
            pos = parts[1]
            pass